In [4]:
import os

In [5]:
%pwd

'/Users/shankii/Documents/EndToEnd_MLOPS/research'

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [15]:
import os
os.chdir("../")
%pwd

'/Users/shankii/Documents/EndToEnd_MLOPS'

In [8]:
# # test_constants.py

# from src.MLOPS_Project.constants import *

# CONFIG_FILE_PATH = Path("config/config.yaml")
# PARAMS_FILE_PATH = Path("params.yaml")
# SCHEMA_FILE_PATH = Path("schema.yaml")

In [ ]:
from src.MLOPS_Project.constants import *
from src.MLOPS_Project.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        ## creating the folder to save the input data
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [17]:
import os
import urllib.request as request
from src.MLOPS_Project import logger
import zipfile

In [18]:
## component-Data Ingestion


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL, filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = (
        config.get_data_ingestion_config()
    )  ## create the root directory as mentioned in the path
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-19 00:04:36,911 INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-19 00:04:36,913 INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-19 00:04:36,914 INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-19 00:04:36,915 INFO: common: created directory at: artifacts]
[2025-03-19 00:04:36,915 INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-19 00:04:36,916 INFO: 284695630: File already exists]
